# Part a

## Import and clean data  

In [ ]:
from util.clean_data import clean_data
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import Ridge
import tensorflow as tf
from tensorflow import keras
import numpy as np
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold

df = pd.read_csv('data/diamond.csv')

# performs interger encoding and log transforms carat and price.
df = clean_data(df)


df = df.drop(["carat", "price"], axis=1)

df = (df - df.mean() ) / df.std(ddof=1)

df.describe()

In [ ]:
# With log price
Y = df['log_price'].values

# with price
#Y = Y_price

feature_names = df.columns

df = df.drop('log_price', axis=1)

X = df.values


N, M = X.shape

## Find the best regulzeration paramaetetr

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

reg_tries = [0, 0.00001, 0.0001, 0.001, 0.01, 0.1, 10, 100, 1000, 5000, 10000]
results = []

for reg_value in reg_tries: 
    ridge_model = Ridge(alpha=reg_value)
    
    scores = cross_val_score(ridge_model, X_train, Y_train, cv=10, scoring="neg_root_mean_squared_error")
    
    mean_rmse = -scores.mean()  # Convert negative RMSE back to positive
    results.append((reg_value, mean_rmse))
    print(f"Regularization: {reg_value:<10} Cross-Validation RMSE: {mean_rmse:.8f}")

reg_values, rmse_values = zip(*results)

# optimal reg model # with lowest RSME
opt_reg = reg_values[rmse_values.index(min(rmse_values))]

# train model with all features and optimal reg we found
model = Ridge(alpha=opt_reg)
model.fit(X_train, Y_train)

y_pred = model.predict(X_test)

# Calculate RMSE for the test set
test_rmse_all_features = np.sqrt(mean_squared_error(Y_test, y_pred))

print("\nFinal model :")
print(f"Test RMSE: {test_rmse_all_features:.8f}")

In [ ]:
# Plot the results
plt.figure(figsize=(10, 6))
plt.plot(reg_values, rmse_values, marker='o', linestyle='-', color='b')
plt.xscale('log')
plt.xlabel('Regularization Strength (alpha)')
plt.ylabel('Cross-Validation RMSE')
plt.title('Effect of Regularization on RMSE')
plt.grid(True)
plt.show()    


## Feature selection:

Backwards selection

In [ ]:
from sklearn.feature_selection import SequentialFeatureSelector

# Backward selection: Start with all features and remove one at a time
backward_selector = SequentialFeatureSelector(
    Ridge(alpha=opt_reg),  # model with optimal reg.
    n_features_to_select="auto",
    direction="backward", 
    cv=10, 
    scoring="neg_root_mean_squared_error"
)
backward_selector.fit(X_train, Y_train)

# Get selected feature indices
selected_backward_features = backward_selector.get_support(indices=True)
print(f"Selected Features (Backward): {selected_backward_features}")


X_train_backward = X_train[:, selected_backward_features]

# backward model evaluate
backward_score = cross_val_score(Ridge(alpha=opt_reg), X_train_backward, Y_train, cv=10, scoring="neg_root_mean_squared_error")

print("Backward score:", -backward_score.mean())


Forward Selection:

In [ ]:
# Forward selection: Start with no features and add one at a time
forward_selector = SequentialFeatureSelector(
    Ridge(alpha=opt_reg),  # model with optimal reg.
    n_features_to_select="auto",
    direction="forward", 
    cv=10, 
    scoring="neg_root_mean_squared_error"
)
forward_selector.fit(X_train, Y_train)

# Get selected feature indices
selected_forward_features = forward_selector.get_support(indices=True)
print(f"Selected Features (Forward): {selected_forward_features}")

X_train_forward = X_train[:, selected_forward_features]

# Forward model evaluation
forward_score = cross_val_score(Ridge(alpha=opt_reg), X_train_forward, Y_train, cv=10, scoring="neg_root_mean_squared_error")

print("Forward score:", -forward_score.mean())


## Final models

In [ ]:
# Train forward model
model_forward = Ridge(alpha=opt_reg)
model_forward.fit(X_train_forward, Y_train)
# Predict and calculate RMSE for forward model
y_pred_forward = model_forward.predict(X_test[:, selected_forward_features])
test_rmse_forward = np.sqrt(mean_squared_error(Y_test, y_pred_forward))

# Train backward model
model_backward = Ridge(alpha=opt_reg)
model_backward.fit(X_train_backward, Y_train)
# Predict and calculate RMSE for backward model
y_pred_backward = model_backward.predict(X_test[:, selected_backward_features])
test_rmse_backward = np.sqrt(mean_squared_error(Y_test, y_pred_backward))


# Calculate R^2 values
r2_all_features = model.score(X_test, Y_test)
r2_forward = model_forward.score(X_test[:, selected_forward_features], Y_test)
r2_backward = model_backward.score(X_test[:, selected_backward_features], Y_test)

# Print R^2 values
print(f"R^2 All features          : {r2_all_features:.8f}")
print(f"R^2 Forward model         : {r2_forward:.8f}")
print(f"R^2 Backward model        : {r2_backward:.8f}")

# Print results
print(f"\nTest RMSE All features          : {test_rmse_all_features:.8f}")
print(f"Test RMSE Forward model         : {test_rmse_forward:.8f}")
print(f"Test RMSE Backward model        : {test_rmse_backward:.8f}")

In [ ]:
print("\nForward Model Feature Weights:")
# print feature weight for all features model
weights = model.coef_
for name, weight in zip(feature_names, weights):
    print(f"Feature: {name:<10} Weight: {weight:.8f}")

# Print feature weights for forward model
print("\nForward Model Feature Weights:")
forward_weights = model_forward.coef_
for name, weight in zip(feature_names[selected_forward_features], forward_weights):
    print(f"Feature: {name:<10} Weight: {weight:.8f}")

# Print feature weights for backward model
print("\nBackward Model Feature Weights:")
backward_weights = model_backward.coef_
for name, weight in zip(feature_names[selected_backward_features], backward_weights):
    print(f"Feature: {name:<10} Weight: {weight:.8f}")

# PART b


# Find optima ann model.

In [ ]:
def find_best_ann_model(x_train, y_train, epochs=10, learning_rate=0.01):
    layer_sizes = [8, 16, 32, 64]
    results = []
    # Define the model

    # trains a one model for each layer size
    for size in layer_sizes:
        print("training with size: ", size)
        
        kf = KFold(n_splits=10, shuffle=True, random_state=42)  # K-Fold setup
        fold_rmse = []  # Store RMSE scores for each fold
        
        for train_idx, val_idx in kf.split(x_train):
            x_train_fold, x_val_fold = x_train[train_idx], x_train[val_idx]
            y_train_fold, y_val_fold = y_train[train_idx], y_train[val_idx]
        
            model = keras.Sequential([
                keras.layers.Dense(size, activation='tanh', input_shape=(x_train.shape[1],)),
                keras.layers.Dense(1) 
            ])
        
            model.compile(optimizer=keras.optimizers.SGD(learning_rate=learning_rate),
                    loss='mse')  # Mean Squared Error for regression
            
            model.fit(x_train_fold, y_train_fold, epochs=epochs, verbose=0)
            
            y_pred = model.predict(x_val_fold)
            rmse = np.sqrt(mean_squared_error(y_val_fold, y_pred))  # Compute RMSE
            fold_rmse.append(rmse)        
            
            
        mean_rmse = np.mean(fold_rmse)  # Average RMSE over all folds
        print(mean_rmse)
        fold_rmse.append((size, mean_rmse))        
    
    best_size, best_rmse = min(results, key=lambda x: x[1])
    print(f"\nBest layer size: {best_size} with RMSE: {best_rmse:.8f}")
    
    
    # Train final model with best layer size
    final_model = keras.Sequential([
        keras.layers.Dense(best_size, activation='tanh', input_shape=(x_train.shape[1],) ),
        keras.layers.Dense(1)
    ])

    final_model.compile(optimizer=keras.optimizers.SGD(learning_rate=learning_rate), loss='mse')
    final_model.fit(x_train, y_train, epochs=epochs, verbose=1)  # Train on full dataset

    return best_size

In [ ]:
#layer = find_best_ann_model(X_train, Y_train, epochs=10, learning_rate=0.01)

In [ ]:

def train_baseline(x_train, y_train):
    # Baseline model: Predicts the mean of the training target values
    mean_value = np.mean(y_train)
    print(f"Baseline model mean value: {mean_value:.8f}")
    return mean_value


def train_ann(x_train, y_train, layers_count, epochs=10, learning_rate=0.01):
    best_size, best_rmse = min(results, key=lambda x: x[1])
    print(f"\nBest layer size: {best_size} with RMSE: {best_rmse:.8f}")
    
    # Train final model with best layer size
    final_model = keras.Sequential([
        keras.layers.Dense(layers_count, activation='tanh', input_shape=(x_train.shape[1],)),
        keras.layers.Dense(1)
    ])

    final_model.compile(optimizer=keras.optimizers.SGD(learning_rate=learning_rate), loss='mse')
    final_model.fit(x_train, y_train, epochs=epochs, verbose=1)  # Train on full dataset

    return final_model
    
    
def train_ridge(x_train, y_train, optimal_reg, feature_selector):
    model = Ridge(alpha=optimal_reg)
    model.fit(x_train, y_train)
    return model
    

def evaluate_model(model, x_test_cv, y_test_cv):
    y_pred_cv = model.predict(x_test_cv)
    # Calculate RMSE
    rmse = np.sqrt(mean_squared_error(y_test_cv, y_pred_cv))
    # Calculate R^2
    r2 = r2_score(y_test_cv, y_pred_cv)
    return rmse, r2
        

In [ ]:

# setup Cross validation K FOLD
kf = KFold(n_splits=10, shuffle=True, random_state=42)

int = 0
for train_index, test_index in kf.split(X_train):
    x_train_cv, x_test_cv = X_train[train_index], X_train[test_index]
    y_train_cv, y_test_cv = Y_train[train_index], Y_train[test_index]
    
    # MODEL 1 ANN model
    ann_model = train_ann(x_train_cv, y_train_cv)
    ann_rmse, ann_r2 = evaluate_model(ann_model, x_test_cv, y_test_cv)
    print(f"RIDGE: RMSE: {ann_rmse:.6f} R2: {ann_r2:.6f}")
    # Model 2 Ridge model from part a
    # Gets index for features from section a:
    selected_features = forward_selector.get_support(indices=True)
    ridge_model = train_ridge(x_train_cv, y_train_cv, opt_reg, selected_features)
    
    ridge_rmse, ridge_r2 = evaluate_model(ridge_model, x_test_cv, y_test_cv)
    print(f"RIDGE: RMSE: {ridge_rmse:.6f} R2: {ridge_r2:.6f}")
    
    ## Model 3: Baseline
    #train_baseline(x_train_cv, y_train_cv)
    
    int += 1
